# Trabajo Práctico 2: Entrenamiento y evaluación de modelos

### Métrica de Performance: Accuracy
Optamos por la eleccíon de esta métrica, debido a que resulta útil en problemas de clasificación. En este caso, estamos frente a un problema de clasificación binaria (gana el equipo 1 o el equipo 2).

### Feature Engineering:
No aplicamos ninguna técnica de feature engineering para mejorar la data de entrada a los modelos, porque los valores posibles de las columnas del set de datos en la mayoria de los casos es: 0/1/2 (ningún equipo/equipo 1/equipo 2) o 0/1 (false/true). Por lo tanto, no pudimos trabajarlas para extraer información extra que nos resulte de utilidad.

In [1]:
import numpy
import pandas
import matplotlib
#import keras
import h5py
import PIL
import seaborn
import sklearn
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Modelos a usar
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import tree

data_lol= pandas.read_csv(r'games_trans.csv', sep=',')

data_lol.head(5)


,Unnamed: 0,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_towerKills,t1_inhibitorKills,...,t2_429,t2ban_429,t2_432,t2ban_432,t2_497,t2ban_497,t2_498,t2ban_498,t2_516,t2ban_516
0,0,1,2,1,1,1,1,2,11,1,...,0,0,0,0,0,0,1,0,0,0
1,1,1,1,1,1,0,1,1,10,4,...,0,0,0,0,0,0,0,0,0,0
2,2,1,2,1,1,1,2,0,8,1,...,0,0,0,0,0,0,0,0,0,0
3,3,1,1,1,1,1,1,0,9,2,...,0,0,0,0,0,0,0,0,0,0
4,4,1,2,1,1,1,1,0,9,2,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# Defino función general, se le pasa Entradas, Target y el Modelo ya creado a entrenar
# Pinta con la meta Accuracy las predicciones para Train y Test
def trainModel(inputs, target, model):
    
    # Separa los datos en train y test
    inputsTrain, inputsTest, targetTrain, targetTest = train_test_split(inputs, target, test_size=0.33, random_state=101)

    # Entrena el modelo 
    model.fit(inputsTrain, targetTrain)
    
    # Hace las predicciones con train y test
    predictionsTrain = model.predict(inputsTrain)
    predictionsTest = model.predict(inputsTest)
    
    # Muestra Accuracy para ambas predicciones
    print('Accuracy Train:',accuracy_score(targetTrain, predictionsTrain))
    print('Accuracy Test:',accuracy_score(targetTest, predictionsTest))
    print('')

In [3]:
# Defino las variables de entrada y el target

#Para Barones y Dragon asesinados
inputsBaronsDragons = data_lol[['t1_baronKills', 't1_dragonKills', 't2_baronKills', 't2_dragonKills']]
targetBaronsDragons = data_lol['winner']

#Para Torres e Inhibidores destruidos
inputsTowersInhibitors = data_lol[['t1_towerKills','t1_inhibitorKills','t2_towerKills','t2_inhibitorKills']]
targetTowersInhibitors = data_lol['winner']

# lista de ids de los campeones
data = []
champions = pandas.read_json('champion_info_2.json')
champion_ids = []
for champion in sorted(champions.data.to_list(), key=lambda k: k['id']):
    champion_ids.append(champion['id'])
champion_ids = champion_ids[1:]

for team in ["1", "2"]:
    for champ_id in champion_ids:
        data.append('t' + team + '_' + str(champ_id))
        data.append('t' + team + 'ban_' + str(champ_id))
        
#Para Campeones elegidos y baneados
inputsChampsAndBans = data_lol[data]
targetChampsAndBans = data_lol['winner']

A) REGRESIÓN LÓGISTICA

In [4]:
# Crea el modelo de Regresión Logistica
logModelBaronsDragons = LogisticRegression()
print('Modelo para Barones y Dragones asesinados')
# Entrena el modelo con los datos de Barones y Dragones asesinados por cada equipo
trainModel(inputsBaronsDragons,targetBaronsDragons,logModelBaronsDragons)

logModelTowersInhibitors = LogisticRegression()
# Entrena el modelo con los datos de Torres e Inhibdores destruidos por cada equipo
print('Modelo para Torres e Inhibidores destruidos')
trainModel(inputsTowersInhibitors,targetTowersInhibitors,logModelTowersInhibitors)

logModelChampsAndBans = LogisticRegression()
# Entrena el modelo con los datos de Campeones elegidos y baneados por cada equipo
print('Modelo para Campeones elegidos y baneados por cada equipo')
trainModel(inputsChampsAndBans,targetChampsAndBans,logModelChampsAndBans)

Modelo para Barones y Dragones asesinados
Accuracy Train: 0.8307148240477709
Accuracy Test: 0.8273305084745762

Modelo para Torres e Inhibidores destruidos
Accuracy Train: 0.9568380775697142
Accuracy Test: 0.9570386064030132

Modelo para Campeones elegidos y baneados por cada equipo


/home/emiliano/Escritorio/ML_LOL/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/emiliano/Escritorio/ML_LOL/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/emiliano/Escritorio/ML_LOL/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy Train: 0.5738303669777958
Accuracy Test: 0.5471398305084746



B) RED NEURONALES MLP

In [5]:
# Crea el modelo de Red Neuronal MLP
MLPModelBaronsDragons= MLPClassifier(activation='relu', solver='adam', hidden_layer_sizes=(50,), random_state=1)
print('Modelo para Barones y Dragons asesinados')
# Entrena el modelo con los datos de Barones y Dragones asesinados por cada equipo
trainModel(inputsBaronsDragons,targetBaronsDragons,MLPModelBaronsDragons)

MLPModelTowersInhibitors = MLPClassifier(activation='relu', solver='adam', hidden_layer_sizes=(50,), random_state=1)
# Entrena el modelo con los datos de Torres e Inhibdores destruidos por cada equipo
print('Modelo para Torres e Inhibidores destruidos')
trainModel(inputsTowersInhibitors,targetTowersInhibitors,MLPModelTowersInhibitors)

MLPModelChampsAndBans= MLPClassifier(activation='relu', solver='adam', hidden_layer_sizes=(20,10),max_iter=10, random_state=1)
# Entrena el modelo con los datos de Campeones elegidos y baneados por cada equipo
print('Modelo para Campeones elegidos y baneados por cada equipo')
trainModel(inputsChampsAndBans,targetChampsAndBans,MLPModelChampsAndBans)


Modelo para Barones y Dragons asesinados
Accuracy Train: 0.8345991071946199
Accuracy Test: 0.8294491525423728

Modelo para Torres e Inhibidores destruidos
Accuracy Train: 0.9615629891587918
Accuracy Test: 0.9615112994350282

Modelo para Campeones elegidos y baneados por cada equipo


/home/emiliano/Escritorio/ML_LOL/venv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Accuracy Train: 0.7101571105571337
Accuracy Test: 0.5314854048964218



En el tercer caso, de Campeones elegidos y baneados, decidimos ir cambiando la capas ocultas y el max_iter, porque con los valores por defecto daba un Acurracy de de 100% de Train, por ende, estaba overfitiando. Lo valores finales que elegimos fueron por pruebas que fuimos haciendo.


C) ARBOLES DE DISICIÓN

In [6]:
# Crea el modelo Árbol de decisión
treeModelBaronsDragons = tree.DecisionTreeClassifier()
print('Modelo para Barones y Dragons asesinados')
# Entrena el modelo con los datos de Barones y Dragones asesinados por cada equipo
trainModel(inputsBaronsDragons,targetBaronsDragons,treeModelBaronsDragons)

treeModelTowersInhibitors = tree.DecisionTreeClassifier()
# Entrena el modelo con los datos de Torres e Inhibdores destruidos por cada equipo
print('Modelo para Torres e Inhibidores destruidos')
trainModel(inputsTowersInhibitors,targetTowersInhibitors,treeModelTowersInhibitors)

treeModelChampsAndBans = tree.DecisionTreeClassifier(max_depth=22)
# Entrena el modelo con los datos de Campeones elegidos y baneados por cada equipo
print('Modelo para Campeones elegidos y baneados por cada equipo')
trainModel(inputsChampsAndBans,targetChampsAndBans,treeModelChampsAndBans)


Modelo para Barones y Dragons asesinados
Accuracy Train: 0.8366861847063598
Accuracy Test: 0.8289194915254238

Modelo para Torres e Inhibidores destruidos
Accuracy Train: 0.9657951185575976
Accuracy Test: 0.9599223163841808

Modelo para Campeones elegidos y baneados por cada equipo
Accuracy Train: 0.6458635283204823
Accuracy Test: 0.5303083804143126



En el tercer caso, de Campeones elegidos y baneados, decidimos ponerle un max_depth de 22, que fuimos probando, ya que en un comienzo sobreentenaba, dando un Accuracy de 100% en train

<h3>Conclusión:</h3>
Decidimos probar con tres set de variables de entradas diferentes, para ver como se comportaban los diferentes modelos. Los sets son, el primero de Barones y Dragones asesinados por cada equipo, el segundo de Torres e Inhibidores destruidos por cada equipo y el tercero de Campeones elegidos y baneados.
Como modelos utilizamos <u>Regresión logística</u>, <u>Red neuronal MLP</u> y <u>Árbol de decisión</u>, los cuales implementamos para cada set de variables.<br>
Para las variables de <u>Barones y Dragones asesinados</u>, observamos que con Regresión Logística, se obtiene un 82% de acurracy, el cual parece ser un valor bastante bueno. porcentaje el cual es similar también para la Red Neuronal y el Árbol, por cual se podría optar por cualquiera de estos modelos.<br>
Para las variables de <u>Torres e Inhibidores destruidos</u>, con Regresión Logística el valor es del 95%, con unas centésimas mayor al accuracy de la predicción de los datos entrenamiento, con lo cual podríamos decir que esta overfitiando un poco. Algo similar sucede con la Red Neuronal pero un un accuracy de 96%, y para el con el Árbol de decisión, el accuracy de Test es una décima menor al de Train, con un 95%, por lo tanto podria decirse que los tres modelos otorgan resultados similares.<br>
Para las variables de <u>Campeones elegidos y baneados</u> por cada equipo, con regresión logística el accuracy da un porcentaje bastante bajo 54%, en cual no serviria para realizar una predicción, ya que seria casi lo mismo que tirar una moneda. Lo mismo sucede con la Red MLP, la cual da un accuracy del 53%, bastante menor al de Test 71%, que un principio overfiteaba, dando un 100%, por lo cual de decidió aplicar algunas de técnicas para que no overfitee tanto, pero no dio buenos resultados en el valor final.<br>
Con el Árbol de decisión, el resultado no difiere mucho de los modelos anteriores, ya que da un 53%. También overftiaba mucho en un principio. Esto tal vez se deba al <u>one hot encoder que realizamos en un comienzo</u>.<br>
Como conclusión de estas variables de entrada, podría decirse, que es muy difícil predecir un resultado simplemente con los campeones elegidos y baneados, ĺo cual tiene algo de sentido.<br>
Para los Barones y Dragones asesinados, se podría obtener un resultado bastante bueno, y para las Torres e Inhibidores destruidos, el resultado es bastante bueno con los tres modelos, con lo cual se podría utilizar.<br>
También comentar, que el dataset contiene una gran cantidad de variables, el cual permitiría hacer muchos análisis y predicciones, dependerá de la persona la cual necesita esta información, saber que es lo que requiere para darle un resultado mas acorde.